IMPORTS

In [ ]:
# math librairies
import numpy as np
from scipy import ndimage

from plotly import express
import pandas               # used with plotly

#3D image processing librairies
import libigl

# custom librairies
import imicpe
import imicpe.optim as optim
print(imicpe.__version__)

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'libigl'